In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from astropy.io import fits
from astropy.visualization import ZScaleInterval
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D
import astropy.units as u

from scipy import stats

import lmfit

NB_wav_Arr = np.arange(455, 855, 10).astype(int)

In [ ]:
vi_cat = fits.open('/home/alberto/almacen/PAUS_data/catalogs/PAUS_LAE_selection_visual_insp_AT.fits')[1].data

vi_cat = vi_cat[~vi_cat['is_junk_VI']  & (vi_cat['lya_NB'] > 0) & (vi_cat['L_lya'] < 49)]
print(len(vi_cat))

In [ ]:
cut_size = 15

cutouts_dir = '/home/alberto/almacen/PAUS_data/cutouts'
lya_img_list =  np.empty((len(vi_cat), cut_size, cut_size))
cont_img_list =  np.empty((len(vi_cat), cut_size, cut_size))

for src in range(len(vi_cat)):
    lya_NB = int(vi_cat['lya_NB'][src])
    ref_id = int(vi_cat['ref_id'][src])
    NB_int_wav = NB_wav_Arr[lya_NB]

    cutout_path = f'{cutouts_dir}/coadd_cutouts/NB{NB_int_wav}/coadd_cutout_{ref_id}.fits'
    cutout = fits.open(cutout_path)
    RA = vi_cat['RA'][src]
    DEC = vi_cat['DEC'][src]
    coords = SkyCoord(RA, DEC, unit='deg')
    wcs = WCS(cutout[0])
    img = cutout[0].data
    cutout_img = Cutout2D(img, coords, size=(cut_size, cut_size),
                          wcs=wcs, mode='partial', fill_value=0.).data

    lya_img_list[src] = cutout_img / np.sum(cutout_img)

    
    # CONT
    NB_Arr_cont = np.concatenate(
        [np.arange(max(0, lya_NB - 6), max(0, lya_NB - 1)),
        np.arange(min(lya_NB + 2, 39), min(lya_NB + 6 + 1, 39))]       
    )
    NB_wav_Arr_cont = NB_wav_Arr[NB_Arr_cont]
    save_coadds_to = f'NB' + '_'.join(NB_wav_Arr_cont.astype(str))

    cutout_path = f'{cutouts_dir}/coadd_cutouts/{save_coadds_to}/coadd_cutout_{ref_id}.fits'
    cutout = fits.open(cutout_path)
    RA = vi_cat['RA'][src]
    DEC = vi_cat['DEC'][src]
    coords = SkyCoord(RA, DEC, unit='deg')
    wcs = WCS(cutout[0])
    img = cutout[0].data
    cutout_img = Cutout2D(img, coords, size=(cut_size, cut_size),
                          wcs=wcs, mode='partial', fill_value=0.).data

    cont_img_list[src] = cutout_img / np.sum(cutout_img)

In [ ]:
stacked_lya_img = np.median(lya_img_list, axis=0)
stacked_cont_img = np.median(cont_img_list, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(9, 5))

[vmin, vmax] = ZScaleInterval(contrast=0.5).get_limits(stacked_lya_img.flatten())
axes[0].imshow(stacked_lya_img, vmin=vmin, vmax=vmax)
[vmin, vmax] = ZScaleInterval(contrast=0.5).get_limits(stacked_cont_img.flatten())
axes[1].imshow(stacked_cont_img, vmin=vmin, vmax=vmax)

plt.show()

In [ ]:
def gaussian_2d(x, y, centerx, centery, sigmax, sigmay, amplitude, c):
    return amplitude * np.exp(-((x - centerx)**2 / (2 * sigmax**2) + (y - centery)**2 / (2 * sigmay**2))) + c

In [ ]:
# Fit Gaussian
pixel_size = 0.4293497

# model = lmfit.models.Gaussian2dModel()
model = lmfit.Model(gaussian_2d, independent_vars=('x', 'y'))

mesh_xxyy = np.meshgrid(np.arange(stacked_lya_img.shape[0]),
                        np.arange(stacked_lya_img.shape[1]))
xx = mesh_xxyy[0].flatten() * pixel_size
yy = mesh_xxyy[1].flatten() * pixel_size

params = lmfit.Parameters()

params.add('centerx', value=3., min=2., max=4)
params.add('centery', value=3., min=2., max=4)
params.add('sigmax', value=2, min=0.01, max=5)
params.add('sigmay', value=2, min=0.01, max=5)
params.add('amplitude', value=0.001, min=0, max=2)
params.add('c', value=5, min=-5, max=10)

result = model.fit(stacked_lya_img.flatten(), x=xx, y=yy, params=params)

lmfit.report_fit(result)


In [ ]:
# Plot to check the fits
fig, ax = plt.subplots()

plot_xx = np.arange(stacked_lya_img.shape[0]) * pixel_size

ax.errorbar(plot_xx, stacked_lya_img[stacked_lya_img.shape[1]//2, :])
# ax.errorbar(plot_xx, stacked_cont_img[stacked_lya_img.shape[1]//2, :])

fit_xx = np.linspace(0, stacked_cont_img.shape[0] * pixel_size, 200)
pdf1 = (stats.norm.pdf(fit_xx, result.params['centerx'], result.params['sigmax'])
        * (result.params['amplitude'] * (((2 * np.pi) ** 0.5 * result.params['sigmax']))) + result.params['c'])
ax.plot(fit_xx, pdf1)


plt.show()


In [ ]:
# Fit Gaussian
pixel_size = 0.4293497

# model = lmfit.models.Gaussian2dModel()
model = lmfit.Model(gaussian_2d, independent_vars=('x', 'y'))

mesh_xxyy = np.meshgrid(np.arange(stacked_cont_img.shape[0]),
                        np.arange(stacked_cont_img.shape[1]))
xx = mesh_xxyy[0].flatten() * pixel_size
yy = mesh_xxyy[1].flatten() * pixel_size

params = lmfit.Parameters()

params.add('centerx', value=3., min=2., max=4)
params.add('centery', value=3., min=2., max=4)
params.add('sigmax', value=2, min=0.001, max=5)
params.add('sigmay', value=2, min=0.001, max=5)
params.add('amplitude', value=0.001, min=0, max=2)
params.add('c', value=5, min=-5, max=10)

result = model.fit(stacked_cont_img.flatten(), x=xx, y=yy, params=params)

lmfit.report_fit(result)


In [ ]:
# Plot to check the fits
fig, ax = plt.subplots()

plot_xx = np.arange(stacked_cont_img.shape[0]) * pixel_size

ax.errorbar(plot_xx, stacked_cont_img[stacked_lya_img.shape[1]//2, :])

fit_xx = np.linspace(0, stacked_cont_img.shape[0] * pixel_size, 200)
pdf1 = (stats.norm.pdf(fit_xx, result.params['centerx'], result.params['sigmax'])
        * (result.params['amplitude'] * (((2 * np.pi) ** 0.5 * result.params['sigmax']))) + result.params['c'])
ax.plot(fit_xx, pdf1)


plt.show()
